# Прогнозирование тепературы стали

## Описание проекта

В целях оптимизации производственных расходов металлургического комбината ООО «Так закаляем сталь» принято решение уменьшить потребление электроэнергии на этапе обработки стали. Необходимо построить модель, которая предскажет температуру стали.

### Описание этапа обработки

Сталь обрабатывают в металлическом ковше вместимостью около 100 тонн. Чтобы ковш выдерживал высокие температуры, изнутри его облицовывают огнеупорным кирпичом. Расплавленную сталь заливают в ковш и подогревают до нужной температуры графитовыми электродами. Они установлены в крышке ковша. 

Из сплава выводится сера (десульфурация), добавлением примесей корректируется химический состав и отбираются пробы. Сталь легируют — изменяют её состав — подавая куски сплава из бункера для сыпучих материалов или проволоку через специальный трайб-аппарат (англ. tribe, «масса»).

Перед тем как первый раз ввести легирующие добавки, измеряют температуру стали и производят её химический анализ. Потом температуру на несколько минут повышают, добавляют легирующие материалы и продувают сплав инертным газом. Затем его перемешивают и снова проводят измерения. Такой цикл повторяется до достижения целевого химического состава и оптимальной температуры плавки.

Тогда расплавленная сталь отправляется на доводку металла или поступает в машину непрерывной разливки. Оттуда готовый продукт выходит в виде заготовок-слябов (англ. *slab*, «плита»).

### Описание данных

Данные состоят из файлов, полученных из разных источников:

- `data_arc.csv` — данные об электродах;
- `data_bulk.csv` — данные о подаче сыпучих материалов (объём);
- `data_bulk_time.csv` — данные о подаче сыпучих материалов (время);
- `data_gas.csv` — данные о продувке сплава газом;
- `data_temp.csv` — результаты измерения температуры;
- `data_wire.csv` — данные о проволочных материалах (объём);
- `data_wire_time.csv` — данные о проволочных материалах (время).

Во всех файлах столбец `key` содержит номер партии. В файлах может быть несколько строк с одинаковым значением `key`: они соответствуют разным итерациям обработки.

#### План работы:  
- Провести исследовательский анализ данных:
    - Посмотреть на распределение исходных данных
    - Проанализировать порядок событий в производственном процессе
    - Исключить партии, которые не имеют последнего замера темпиратуры, либо первый замер соответствует последнему, либо замеры отсутствуют 
    - Оставить только события, которые находятся в промежутке между первым и последним замерами
- Подобрать признаки, на которых будет обучаться модель:
    - Сгенерировать ряд признаков
    - Проверить признаки на мультиколлинеарность, исключить лишние
    - Провести масштабирование признаков
- Построить модель, которая предскажет температуру стали:
    - Сформировать обучающую, валидационную и тестовую выборки
    - Выбрать несколько моделей и проверить их эффективность на валидационной выборке с помощью показателя эффективности MAE
    - Эффективность лучшей модели проверить на тестовой выборке

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

from sklearn.metrics import mean_absolute_error as mae

In [ ]:
arc = pd.read_csv('/datasets/final_steel/data_arc.csv')
bulk = pd.read_csv('/datasets/final_steel/data_bulk.csv')
bulk_time = pd.read_csv('/datasets/final_steel/data_bulk_time.csv')
gas = pd.read_csv('/datasets/final_steel/data_gas.csv')
temp = pd.read_csv('/datasets/final_steel/data_temp.csv')
wire = pd.read_csv('/datasets/final_steel/data_wire.csv')
wire_time = pd.read_csv('/datasets/final_steel/data_wire_time.csv')

## Исследовательский анализ данных

In [ ]:
try:
    arc = pd.read_csv('data_arc.csv', parse_dates=['Начало нагрева дугой', 'Конец нагрева дугой'], dayfirst=True)
    bulk = pd.read_csv('data_bulk.csv')
    bulk_time = pd.read_csv('data_bulk_time.csv', parse_dates=['Bulk 1','Bulk 2', 'Bulk 3',
                                                                                'Bulk 4','Bulk 5','Bulk 6',
                                                                                'Bulk 7', 'Bulk 8','Bulk 9',
                                                                                'Bulk 10','Bulk 11','Bulk 12',
                                                                                'Bulk 13','Bulk 14','Bulk 15'])
    gas = pd.read_csv('data_gas.csv')
    temp = pd.read_csv('data_temp.csv', parse_dates=['Время замера'])
    wire = pd.read_csv('data_wire.csv')
    wire_time = pd.read_csv('data_wire_time.csv', parse_dates=['Wire 1','Wire 2', 'Wire 3',
                                                                                'Wire 4','Wire 5','Wire 6',
                                                                                'Wire 7', 'Wire 8','Wire 9'],)
except:
    arc = pd.read_csv('/datasets/final_steel/data_arc.csv', parse_dates=['Начало нагрева дугой', 'Конец нагрева дугой'])
    bulk = pd.read_csv('/datasets/final_steel/data_bulk.csv')
    bulk_time = pd.read_csv('/datasets/final_steel/data_bulk_time.csv', parse_dates=['Bulk 1','Bulk 2', 'Bulk 3',
                                                                                'Bulk 4','Bulk 5','Bulk 6',
                                                                                'Bulk 7', 'Bulk 8','Bulk 9',
                                                                                'Bulk 10','Bulk 11','Bulk 12',
                                                                                'Bulk 13','Bulk 14','Bulk 15'],
                                                                                 dayfirst=True)
    gas = pd.read_csv('/datasets/final_steel/data_gas.csv')
    temp = pd.read_csv('/datasets/final_steel/data_temp.csv', parse_dates=['Время замера'])
    wire = pd.read_csv('/datasets/final_steel/data_wire.csv')
    wire_time = pd.read_csv('/datasets/final_steel/data_wire_time.csv', parse_dates=['Wire 1','Wire 2', 'Wire 3',
                                                                                'Wire 4','Wire 5','Wire 6',
                                                                                'Wire 7', 'Wire 8','Wire 9'])

Рассмотрим подробно представленные данные

Посмотрим на данные по электродам, таблица `data_arc.csv`

In [ ]:
display(arc.sample(10))
display(arc.info())
display(arc.describe())
arc.duplicated().sum()

В таблице 14876 записей о временых точках начала и конца нагрева стали и мощность. Мощность представлена в виде активной и реактивной мощностей.


Реактивная мощность связана с полной мощностью S и активной мощностью P соотношением:
$$Q = \sqrt{S^2-P^2}$$

Полная мощность — величина, равная произведению действующих значений периодического электрического тока I в цепи и напряжения U на её зажимах S=U * I связана с активной и реактивной мощностями соотношением:

$$S = \sqrt{P^2+Q^2}$$
где P — активная мощность, Q — реактивная мощность.



Так как в энергозатратах участвует полная мощность, будем использовать её как обучающий признак.

Есть отрицательное значение реактивной мощности с очень высоким значением по модулю, посмотрим на данные всей партии.
Дубликаты отсутствуют.

In [ ]:
arc[arc['Реактивная мощность'] < 0]

In [ ]:
arc[arc['key'] == 2116]

Похоже на ошибку либо был скачок напряжения, который выдал такой результат, в любом случае, эту партию (2116) стоит исключить из дальнейшего рассмотрения на этапе предобработки данных чтобы не искажать картину.

Рассмотрим данные об объеме сыпучих материалов, таблица `data_bulk.csv`

In [ ]:
display(bulk.sample(10))
display(bulk.info())
display(bulk.describe())
bulk.duplicated().sum()

В таблице 3129 записей об объеме добавления 15 видов сыпучих материалов, много пропусков, они связаны с тем, что исходя из химического анализа пробы, необходимо добавить не обязательно все элементы, а только те, которых нехватает до необходимого химического состава сплава, пропуски заменим нулями.

Отрицательных значений нет, дубликатов тоже нет.

Рассмотрим данные о времени добавления сыпучих материалов, таблица `data_bulk_time.csv`

In [ ]:
display(bulk_time.sample(10))
display(bulk_time.info())
display(bulk_time.describe())
bulk_time.duplicated().sum()

В таблице 3129 записей о времени добавления 15 видов сыпучих материалов, много пропусков, они связаны с тем, что исходя из химического анализа пробы, необходимо добавить не обязательно все элементы, а только те, которых нехватает до необходимого химического состава сплава.

Отрицательных значений нет, дубликатов тоже нет.

Вероятно, эта информация для нас лишняя, поскольку мы будем работать с партиями и фактор точного времени для нас не  важен.

Рассмотрим данные о продувке сплава газом, таблица `data_gas.csv`

In [ ]:
display(gas.sample(10))
display(gas.info())
display(gas.describe())
gas.duplicated().sum()

В таблице 3239 записей об объеме добавления газа, пропусков, отрицательных значений, дубликатов нет.

Рассмотрим данные о замерах температуры, таблица `data_temp.csv`, температура последнего замера - целевой признак.

In [ ]:
display(temp.sample(10))
display(temp.info())
display(temp.describe())
temp.duplicated().sum()

В таблице 15907 записей об измерениях температуры, при этом присутствуют прпуски в замерах, всего 13 006 известных значений температуры. 

Отрицательные значения и дубликаты отсутствуют.

Попробуем выяснить с чем связаны пропуски.

In [ ]:
temp[temp.isna().any(axis=1)]

Интересно, пропуски начались с 2500 партии, посмотрим на количество замеров температуры в каждой партии.

In [ ]:
temp_count = temp.pivot_table(index='key', values='Температура',aggfunc='count')

temp_count.plot(figsize=(18,6))
plt.title('Количество замеров температуры в партии')
plt.xlabel('Партия')
plt.ylabel('Количество замеров');


Начиная с 2500 партии был только один замер температуры на партию, выборочно посмотрим на партии с одним замером.

In [ ]:
temp[temp['key'].isin([2501])]

In [ ]:
temp[temp['key'].isin([2749])]

In [ ]:
temp[temp['key'].isin([3004])]

In [ ]:
temp[temp['key'].isin([3102])]

In [ ]:
temp[temp['key'].isin([3239])]

Видим, что начиная с 2500 партии есть данные замера только в первой итерации. Проверим, что это действительно так для всех  этх партий, заполним пропуски нулями.

In [ ]:
temp['Температура'] = temp['Температура'].fillna(0)

Поскольку целевым признаком является температура последнего замера в партии нас интересует именно она. Если после 2500 партии действительно не было замеров в последней итерации, то на графике после 2500 партии мы увидим нули.

In [ ]:
temp_last = temp.pivot_table(index='key', values='Температура',aggfunc='last')
temp_last.plot(figsize=(18,6))
plt.title('Последняя температура в партии')
plt.xlabel('Партия')
plt.ylabel('Температура');

Гипотеза подтвердилась, после 2500 партии только нули, значит, в этих партиях нет необходимо целевого признака и на этих данных обучать модель мы не можем. На этапе предобработки удалим эти данные.

В данных имеются замеры с температурой ниже плавления стали (1350°С)

In [ ]:
temp['Температура'].plot(figsize=(16,6))
plt.title('Динамика замеров температуры')
plt.xlabel('Замер')
plt.ylabel('Температура');

На графике хорошо видны выбросы с нулевой температурой замера(партии начиная с 2500), а также несколько замеров со слишком низкой для плавления стали температурой - возможно это как-то связано с простоем стали перед замером, когда сталь уже остыла или с браком датчика температуры, в любом случае, эти аномалии будут мешать в дальнейшей работе.

Рассмотрим данные об объеме добавления проволочных материалов, таблица data_wire_.csv

In [ ]:
display(wire.sample(10))
display(wire.info())
display(wire.describe())
wire.duplicated().sum()

В таблице 3081 запись об объеме добавления 9 видов проволочных материалов, имеется большое количество пропусков ввиду анализа химического состава и выбора конкретных материалов для улучшения качества стали, пропуски заменим нулями.

Рассмотрим данные о времени добавления проволочных материалов, таблица `data_wire_time.csv`

In [ ]:
display(wire_time.sample(10))
display(wire_time.info())
display(wire_time.describe())
wire_time.duplicated().sum()

В таблице 3081 запись о времени добавления 9 видов проволочных материалов, имеется большое количество пропусков ввиду анализа химического состава и выбора конкретных материалов для улучшения качества стали, равно как и для сыпучих материалов точное время добавления проволоки для нас не представляет ценности

### Вывод: 
Проведён исследовательский анализ предоставленных данных:
- `data_arc.csv` — данные об электродах
- `data_bulk.csv` — данные о подаче сыпучих материалов (объём)
- `data_bulk_time.csv` — данные о подаче сыпучих материалов (время)
- `data_gas.csv` — данные о продувке сплава газом
- `data_temp.csv` — результаты измерения температуры
- `data_wire.csv` — данные о проволочных материалах (объём)
- `data_wire_time.csv` — данные о проволочных материалах (время)

Изучены данные о времени и мощности нагрева электродами, об объеме и времени добавления дополнительных материалов для улучшения качества стали, о продувке газом и результатах измерения температур. В таблицах данные о нескольких итерациях для одной партиии, что обусловлено процессом получения необходимого химического состава стали и температуры её плавления, имеются пропуски и некорректные данные.

Для дальнейшей работы необходимо провести предварительную обработку данных:

- `data_arc.csv` — удалить партию 2116 с некорректным значением реактивной мощности, вычислить полную мощность и использовать её в качестве обучающего признака.
- `data_gas.csv` — вероятно, количество поданного газа может влиять на температуру.
- `data_temp.csv` — удалить партии начиная с 2500 за неимением целевого признака, удалить замеры со значением ниже температуры плавления стали(1350°С).
- `data_bulk.csv` — вероятно, количество поданного материала может влиять на температуру.
- `data_bulk_time.csv` — не представляет ценности для обучения модели.
- `data_wire.csv` вероятно, количество поданного материала может влиять на температуру.
- `data_wire_time.csv` — не представляет ценности для обучения модели.

В данных об объемах присадок и продувке газом:
- заполнить пропуски нулями;
- сделать столбец 'key' индексом - потому что все данные будут объединяться по номеру партии.

Так же необходимо преобразовать названия признаков в соответствии с правилами стиля.

## Предобработка данных

`arc - данные об электродах`

Переименуем названия признаков

In [ ]:
arc = arc.rename(columns = {
    'Начало нагрева дугой':'start_arc_heating', 
    'Конец нагрева дугой' : 'end_arc_heating', 
    'Активная мощность' : 'active_power',
    'Реактивная мощность' : 'reactive_power'}) 

Удаляем из набора данных партию с аномальной реактивной мощностью и партии без целевого признака.

In [ ]:
arc = arc[(arc['reactive_power'] >0) & (arc['key']<2500)]
arc.info()

Посчитаем значения полной мощности.

In [ ]:
arc['full_power'] = (arc['active_power']**2 + arc['reactive_power']**2).apply(np.sqrt)

Посмотрим на распределения признаков, создадим функцию, чтобы в дальнейшем исследовать распределения и других признаков

In [ ]:
def plot(df,col, title):
    
    sns.set_style('whitegrid')
    ax = plt.subplots(figsize = (16,6))
    
    q_1 = list(df[col].quantile([0.25]))[0]
    q_3 = list(df[col].quantile([0.75]))[0]
    
    chart = sns.histplot(data=df[col]).set_xlim(0, q_3 + 2.5*(q_3 - q_1))
    plt.title(title)
    plt.show()
    
    chart_2 = sns.boxplot(data=df, y=col)
    plt.title(title)
    
    display(df[col].describe());

In [ ]:
 plot(arc,'active_power', 'Распределение значений активной мощности')

Распределение значений активной мощности смещено вправо. Взглянем на реактивную мощность.

In [ ]:
plot(arc,'reactive_power', 'Распределение значений реактивной мощности')

Распределение значений активной мощности смещено вправо, то есть, у нас наблюдались скачки в мощности. Посмотрим на полную мощность.

In [ ]:
plot(arc,'full_power', 'Распределение значений полной мощности')


Распределение полной мощности ожидаемо повторяет распределение активной и реактивной мощностей.

Целесообразно изучить суммарное время нагрева за каждую партию

In [ ]:
arc[['start_arc_heating', 'end_arc_heating']] = arc[['start_arc_heating', 
                                                                'end_arc_heating']]

arc['batch_heating_time'] = (arc['end_arc_heating'] - 
                               arc['start_arc_heating']).dt.total_seconds()

arc

In [ ]:
arc_time = arc.pivot_table(index='key', values='batch_heating_time',aggfunc='sum')
arc_time

Посчитаем сколько раз каждая партия подвергалась нагреву и сведём эти данные в таблицу по партиям.

In [ ]:
arc_count_iter = arc.pivot_table(index='key', values='start_arc_heating',aggfunc='count')
arc_count_iter_time = arc_count_iter.join(arc_time)
arc_count_iter_time.columns = ['count_iter','batch_heating_time']
arc_count_iter_time

In [ ]:
plot(arc_count_iter_time,'count_iter', 'Распределение количества нагревов в партии')

Распределение похоже на нормальное, немного смещено вправо - среднее превышает медиану. Большая часть значений лежит в области 3-6 итераций нагрева.

In [ ]:
plot(arc_count_iter_time,'batch_heating_time', 'Распределение общего времени нагрева партии')

Распределение общего времени нагрева партии похоже на нормальное - медиана и средняя почти совпадают, в среднем нагрев происходил около 13 минут.

Исходя из того, что общее время нагрева и количество итераций нагрева могут служить признаками для обучения, добавим еще один - полную среднюю мощность за партию и далее проверим, как эти признаки связаны между собой.

In [ ]:
arc_count_iter_time = arc_count_iter_time.join(arc.pivot_table(index='key',values='full_power',aggfunc='mean'))
arc_count_iter_time

Посмотрим на корреляцию созданных признаков.

In [ ]:
arc_count_iter_time.corr()

Очевидно, что время нагрева и активная мощность заметно коррелируют, к тому же время нагрева и количество итераций также достаточно сильно коррелируют, поэтому отбросим признак со временем нагрева.

In [ ]:
arc_count_iter_time = arc_count_iter_time.drop('batch_heating_time', axis=1)
arc_count_iter_time

`temp - результаты измерения температуры`

Переименуем названия признаков.

In [ ]:
temp = temp.rename(columns = {
    'Время замера':'time_check', 
    'Температура' : 'temperature'}) 

Исключим аномально низкие значения температуры и партии до 2500 где доступно только первое измерение -  эти данные, мы не сможем использовать при обучении модели.

In [ ]:
temp = temp[(temp['temperature'] > 1349) & (temp['key'] <= 2499)]
temp.info()

Выделим первую и последнюю измеренную температуру в партии.

In [ ]:
temps = temp.pivot_table(index='key', values='temperature', aggfunc=['first','last'])
temps.columns = ['first_temp', 'last_temp']
temps.info()

Посчитаем длительность между первым и последним замером температуры в секундах.

In [ ]:
temp_time = temp.pivot_table(index='key', values='time_check', aggfunc=['first','last','count'])
temp_time.columns = ['time_first_temp', 'time_last_temp', 'count_check']
temp_time[['time_first_temp', 'time_last_temp']] = temp_time[['time_first_temp', 
                                                                'time_last_temp']]

temp_time['interval'] = (temp_time['time_last_temp'] - 
                               temp_time['time_first_temp']).dt.total_seconds()

temp_time

Посмотрим на распределение времени между первым и последним замерами.

In [ ]:
plot(temp_time,'interval', 'Распределение времени между первым и последним замерами')

Распределение смещено вправо, средний интервал почти на четыре минуты длиннее медианного. 

`data_bulk.csv` — данные о подаче сыпучих материалов (объём)

`data_wire.csv` — данные о проволочных материалах (объём)

`data_gas.csv` — данные о продувке сплава газом

Переименуем названия признаков.

In [ ]:
bulk = bulk.rename(columns = {
    'Bulk 1' : 'bulk_1', 'Bulk 2' : 'bulk_2',
    'Bulk 3' : 'bulk_3', 'Bulk 4' : 'bulk_4',
    'Bulk 5' : 'bulk_5', 'Bulk 6' : 'bulk_6',
    'Bulk 7' : 'bulk_7', 'Bulk 8' : 'bulk_8',
    'Bulk 9' : 'bulk_9', 'Bulk 10' : 'bulk_10',
    'Bulk 11' : 'bulk_11', 'Bulk 12' : 'bulk_12',
    'Bulk 13' : 'bulk_13', 'Bulk 14' : 'bulk_14',
    'Bulk 15' : 'bulk_15'})

In [ ]:
wire = wire.rename(columns = {
    'Wire 1' : 'wire_1', 'Wire 2' : 'wire_2',
    'Wire 3' : 'wire_3', 'Wire 4' : 'wire_4',
    'Wire 5' : 'wire_5', 'Wire 6' : 'wire_6',
    'Wire 7' : 'wire_7', 'Wire 8' : 'wire_8',
    'Wire 9' : 'wire_9'})


In [ ]:
gas = gas.rename(columns = {
    'Газ 1' : 'gas_1'})

На этапе исследовательского анализа, мы убедились, что в данных о дополнительных материалах нет отрицательных значений. Данные о проволоке и сыпучих материалах имеют много признаков, проверять каждый мы не будем. Посмотрим на распределение данных о продувке газом.

In [ ]:
plot(gas,'gas_1', 'Распределение объема газа')

В целом можно сказать, что распределение похоже на нормальное, есть моменты, когда газа поступало относительно много. 
Заполним пропуски в признаках нулями и сделаем столбец 'key' индексом.

In [ ]:
def fill_and_index(df):
    
    df = df.fillna(0).set_index('key')
    return df

In [ ]:
gas, bulk, wire = fill_and_index(gas), fill_and_index(bulk), fill_and_index(wire)

**Объединим признаки в одну таблицу**

- temps - температура первого и последнего (целевой признак) замеров
- temp_time - время между первым и последним замером
- arc_count_iter_time - средняя на партию полная мощность и количество нагревов
- gas, wire, bulk - данные о присадках и продуве газом

In [ ]:
data = temps.join([temp_time[['interval']], arc_count_iter_time, gas, wire, bulk])
data

In [ ]:
data.info()

In [ ]:
data[data.isna().any(axis=1)]

После объединения таблиц получились пропуски. Вероятно, это связано с тем, что во вспомогательных данных отсутствовали партии, которые были в данных об измерении температуры.

Удалим две строки с пропусками в полной мощности, остальные пропуски заполним нулями.

In [ ]:
data.dropna(subset=['full_power'],inplace=True)

In [ ]:
data = data.fillna(0)
data.info()

Построим тепловую карту корреляции признаков

In [ ]:
data_corr = data.corr()

In [ ]:
plt.figure(figsize=(20,15))
sns.heatmap(data_corr, annot=True, fmt='.2g', vmin=-1, vmax=1, center= 0, cmap= 'coolwarm',
        xticklabels=data_corr.columns,
        yticklabels=data_corr.columns)

Все что имеет коэффициентт корреляции 0,9 и более исключим:wire_8, bulk_9  и заоодно wire_5 - ни на что не влияет.

In [ ]:
data.drop(['wire_8', 'bulk_9', 'wire_5'], axis='columns', inplace=True) 
data

### Вывод:

Проведена предварительная обработка данных:

- Преобразованы названия признаков в соответствии с правилами стиля.
- Удалены некорректные данные в признаках  вычислена полная мощность и добавлена качестве обучающего признака.
- В данных об объемах присадок и продувке газом пропуски заполнены нулями.
- Произведено объединение обучающих признаков в одну таблицу с признаками. 
- Построена тепловая карта корреляции признаков и удалены признаки с коэффициентом корреляции 0.7 и выше.

Данные подготовлены для дальнейшего разбиения на выборки для построения и обучения моделей.

**План работы:**
1. Исследовательский анализ полученных данных. Корректировка задачи.
2. Предобработка данных для создания признаков модели.
3. Создание, обучение, тестирование модели.
4. Выводы.

**Вопросы:**
- На каком этапе необходимо предсказать температуру расплавленной стали? - ответ получен на консультации - необходимо предсказать последнюю замеренную температуру.
- Есть ли какие либо дополнительные, не выданные  нам данные, которые помогли бы улучшить качество модели?
- На консультации обсуждалась возможность суммировать количество всех присадок по партиям, я не стала этого делать ввиду того что температурра плавления разных присадок может быть разная,соответственно, каждая присадка может влиять на итоговое повышение температуры. Верный ли такой вывод?
- При некоторых размышлениях, как признак мощности была выбрана полная мощность, ведь именно она характеризует количество потребляемой энегрии
- Как мне кажется, хорошим признаком мог бы являться химический состав сплава с температурой плавления для каждой присадки, точнее его отклонение от требуемого, так как именно от него зависит, будут ли добавлять присадки, сколько и каких, будут ли нагревать сплав повторно, сколько раз и как долго будет длиться нагрев. Есть у нас такие данные?

## Обучение моделей

Подготовим данные. 

Выделим признаки и целевой признак.



In [ ]:
target = data['last_temp']
features = data.drop('last_temp',axis=1)

Разделение выборок.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=12345)
X_train_valid, X_valid, y_train_valid, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=12345)

print('Valid shapes:')
print(X_train_valid.shape, X_valid.shape, y_train_valid.shape, y_valid.shape)
print('Train/test shapes:')
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Перед обучением моделей узнаем, какую точность дает константная модель. Константная модель выбирается так, чтобы значение метрики MAE было предельно низким. Нужно найти такое значение **a**, при котором достигается минимум: $$ MAE=1/N\displaystyle\sum_{i=1}^{N} |y_i - a| $$
Минимум получается, когда **a** равно медиане целевого признака. Возьмем медианную последнюю температуру обучающей выборки и предскажем ей целевой признак тестовой выборки.

In [ ]:
dummy_mae = mae(y_test, [np.median(y_train) for i in y_test])
print(f'Точность константной модели {dummy_mae:.2f}')

Посмотрим на стандартное отклонение целевого признака.

In [ ]:
data['last_temp'].std()

Ошибка константной модели ниже стандартного отклонения.

Создадим функцию обучения моделей и вывода метрики MAE.

In [ ]:
def model_result(model, X_train, y_train, X_test, y_test):
        
    model.fit(X_train, y_train)  
      
    y_pred = model.predict(X_test)  

    return mae(y_test, y_pred)

Cначала обучим модель через алгоритм линейной регрессии.

In [ ]:
model_linear = LinearRegression()
model_linear.fit(X_train, y_train)
mae_linear = mae(y_test, model_linear.predict(X_test))
mae_linear

Модель показала более высокую точность, по сравнению с константной моделью. Ошибка при этом в 2 раза меньше стандартного отклонения.

Посмотрим на результат обучения гребневой модели.

In [ ]:
result_ridge=[]
for i in np.arange(0.1, 1.5, 0.25):
    model_ridge = Ridge(random_state=12345, alpha=i)
    result_ridge.append(model_result(model_ridge, X_train_valid, y_train_valid, X_valid, y_valid))
result_ridge

In [ ]:
pd.DataFrame(data=result_ridge,
             index=np.arange(0.1, 1.5, 0.25),
             columns=['score'])

In [ ]:
model_ridge = Ridge(random_state=12345, alpha=1.35)
model_ridge.fit(X_train, y_train)
mae_ridge = mae(y_test, model_ridge.predict(X_test))
mae_ridge

Результат гребневой регрессии совсем немного лучше линейной, при этом с увеличением значения alpha незначительно улучшается метрика

Обучим случайный лес.

In [ ]:
result_forest=[]
for i in range(1, 101, 5):
    model_forest = RandomForestRegressor(random_state=12345, n_estimators=i, max_depth = 11, n_jobs=-1, max_features = "sqrt")
    result_forest.append(model_result(model_forest, X_train_valid, y_train_valid, X_valid, y_valid))
result_forest

In [ ]:
pd.DataFrame(data=result_forest,
             index=range(1, 101, 5),
             columns=['score'])

In [ ]:
model = RandomForestRegressor(random_state=12345, n_estimators=86, max_depth = 11, n_jobs=-1, max_features = "sqrt")
model.fit(X_train, y_train)
mae_rf = mae(y_test, model.predict(X_test))
mae_rf

Случайный лес показывает немногим лучший результат чем гребневая и линейная регрессии, но до требуемого менее 6.8 не дотягивает.

Попробуем градиентную модель CatBoost.

In [ ]:
model_cb = CatBoostRegressor(loss_function = 'MAE', verbose=100, random_state=12345)
model_cb.fit(X_train, y_train)
mae_cb = mae(y_test, model_cb.predict(X_test))
mae_cb

CatBoost по умолчанию выдал точность 6.01. По опыту коллег известно, что настройка CatBoost редко позволяет существенно увеличить качество. Поэтому оставим модель на настройках по умолчанию.

Взглянем на важность признаков.

In [ ]:
importances = pd.DataFrame({'feature':X_train.columns,'importance':np.round(model_cb.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False).set_index('feature')
print(importances)
importances.plot.bar();

Как видно из графика выше наибольше влияние на конечную температуру оказывают первая измеренная температура, полная мощность и добавление проволочного материала 1. Наименьшее влияние оказывают сыпучие материалы 13, 8, 2 и проволочные материалы 7 и 9.

Сравним качество моделей.

In [ ]:
overall = pd.DataFrame(data=[mae_linear, mae_ridge, mae_rf, mae_cb, dummy_mae],
                       columns=['MAE на тестовой выборке'],
                      index=['Линейная_регрессия','Гребневая регрессия','Случайный_лес', 'CatBoost', 'Константная модель'])
overall.sort_values('MAE на тестовой выборке')

Отобразим на графике точность предсказаний лучшей модели.

In [ ]:
plt.figure(figsize=(15, 5))

plt.plot(y_test, 'o', label='True')
plt.plot(y_test.index, model_cb.predict(X_test), 'o', label='Predict')

plt.title('Test prediction')
plt.legend()
plt.show()

### Вывод:
На этапе обучения моделей было произведено разбение на выборки: `train`, `test`, `valid`.

Обучены модели :
- Линейная регрессия
- Гребневая регрессия
- Случайный лес
- Градиентный бустинг

Лучшее качество показала модель градиентного бустинга, MAE 6.01. При этом наибольше влияние на конечную температуру оказывают:
 - первая измеренная температура
 - полная мощность
 - добавление проволочного материала 1.
 
Добавление остальных легирующих материалов незначительно либо совсем не оказывают влияния на конечную температуру.

Точность предсказаний наглядно отображена на графике.

## Общий вывод:
В рамках работы над задачей было сделано следующее:
- Проведён исследовательский анализ предоставленных данных. Изучены данные о времени и мощности нагрева электродами, об объеме и времени добавления дополнительных материалов для улучшения качества стали, о продувке газом и результатах измерения температур. Данные о времени добавления легирующих материалов сочтены не существенными и исключены из дальнейшего рассмотрения.

- Проведена предварительная обработка данных, удалены аномалии(большая часть данных целевого признака отсутствовала), исправлены некорректные данные, произведён отбор обучающих признаков и создание дополнительных признаков, итоговый набор признаков объедиен в общую таблицу и  откорректирован при помощи тепловой карты, отброшены признаки с корреляцией выше 0.9.

- Созданы и обучены четыре модели:
    - Линейная регрессия
    - Гребневая регрессия
    - Случайный лес
    - Градиентный бустинг

Все обученные модели разумны - показали качество выше, чем у константной.

Путём сравнения результатов точности, модель Градиентный бустинг была определена как лучшая с MAE = 6.01

Как видно из графика, модель предсказывет температуру довольно близко к факту, хотя есть и некоторые расхождения. Есть значения факта с высокой температурой или наоборот низкой, а предсказания показывают средний результат.

В среднем модель ошибается на 6.01 градусов (меньше 0,4% от средней температуры)

Данная модель может быть использована для предсказания конечной температуры сплава после всех манипуляций с ним с погрешностью до 6 градусов цельсия. 
Модель можно использовать для поиска оптимальных конфигураций начального нагрева и мощности нагрева сплава для уменьшения использования электроэнергии.
    

## Итоговый отчёт

1. В ходе решения поставленной задачи выполнено следующее:
 - Проведен исследовательский анализ данных:
    - рассмотрено на распределение исходных данных
    - проанализирован порядок событий в производственном процессе
    - исключены партии, которые не имеют целевого признака
    - исключены лишние наборы данных 
    - оставлены только те события, которые находятся в промежутке между первым и последним замерами
    
- Подобраны и созданы дополнительные признаки для обучения модели:
    - сгенерирован ряд признаков
    - признаки проверены на мультиколлинеарность, исключены лишние
    
    
- Построить модель, которая предскажет температуру стали:
    - сформированы обучающая, валидационная и тестовая выборки
    - выбраны несколько моделей:
        - Линейная регрессия
        - Гребневая регрессия
        - Случайный лес
        - Градиентный бустинг
    - проверена эффективность моделей на валидационной выборке с помощью показателя эффективности MAE
    - эффективность лучшей модели проверена на тестовой выборке
    
2. Дополнительно было проведено масштабирование признаков, однако модели показали лучший результат без масштабирования, поэтому принято решение опустить этот пункт.

3. Ключевыми шагами в решении задачи хочется выделить детальную подготовку данных, создание новых важных признаков, исключение лишних признаков и применение градиентной модели.

4. Итоговой моделью стала модель с наилучшим результатом точночти предсказаний - CatBoost  c погрешностью в 6°C. Точность модели можно улучшить имея больше данных, например, эталонный химический состав итогового сплава и/или зная некоторые процессы "внутренней кухни" обусловленные человеческими факторами.
    